In [ ]:
import numpy as np
import pandas as pd
import plotnine as p9
import matplotlib.pyplot as plt
import sympy as sp
import re

from random import uniform as rand
from matplotlib import cm
from numpy import array, transpose, linspace, sum, exp, log
from numpy.linalg import inv
from pandas import DataFrame as df
%matplotlib inline

# Style definitions
plt.style.use('dark_background')
p9.theme_set(
    p9.theme_dark() + 
    p9.theme(rect=p9.element_rect(color='black', size=3, fill='black')) +
    p9.theme(text=p9.element_text(color='lightgray')))
COLORS = {'red': '#ff2626',
          'orange': '#f29524',
          'green': '#169340',
          'blue': '#377CB9',
          'white': '#EEEEEE',
          'black': '#222222'}

# Misc utils
def squaremesh(low,high,res):
    return np.meshgrid(np.linspace(low,high,res),np.linspace(low,high,res))
def randspace(low, high, res):
    return array([rand(low, high) for _ in range(res)])
def field(f, low, high, res):
    X, Y = squaremesh(low, high, res)
    return [X, Y, f(X, Y)]
def unbox(dictionary, *items):
    return [dictionary[item] for item in items]

# Plot functions
def ax3D():
    ax = plt.axes(projection='3d', facecolor=(0,0,0,0))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    return ax  
def ax2D():
    ax = plt.axes(facecolor=(0,0,0,0))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    return ax  
def darkgraph():
    return plt.figure(figsize=(20,10), facecolor=(0.1,0.1,0.1))
def surface(X, Y, Z, fig, ax):
    surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0)
    fig.colorbar(surf, shrink=0.6, aspect=5)
def translucent(X, Y, Z, fig, ax, res=300, alpha=0.6):
    surf = ax.contourf(X, Y, Z, res, cmap=cm.coolwarm, linewidth=0, alpha=alpha)
    fig.colorbar(surf, shrink=0.5, aspect=5)
def fitplot2D(X, Y, f, res=50):
    if len(X)!=len(Y): raise
    d1 = df({'x':X,'y':Y,'Source':['Original']*len(X)})
    Xf = linspace(min(X),max(X),res)
    d2 = df({'x':Xf,'yf':f(Xf),'Source':['Fitted']*len(Xf)})
    return (p9.ggplot(pd.concat([d1,d2])) +
            p9.geom_point(p9.aes('x', 'y', color = 'Source')) +
            p9.geom_path(p9.aes('x', 'yf', color = 'Source')))
def fitplot3D(X, Y, Z, f, fig, ax, res=50, alpha=0.2, pcolor='black'):
    if not len(X)==len(Y)==len(Z): raise
    Xf,Yf = squaremesh(min(X),max(X),res)
    Zf = array([f(x,y) for x,y in zip(np.ravel(Xf), np.ravel(Yf))])
    Zf = Zf.reshape(Xf.shape)
    translucent(Xf,Yf,Zf,fig,ax,res=300,alpha=0.2)
    ax.scatter(X, Y, Z, s=9, color=pcolor)

# Ignore warnings (keep commented during editting)
from warnings import filterwarnings
filterwarnings('ignore')

# Clear this cell's output
from IPython.display import clear_output
clear_output()

Physics Institute of São Carlos ([IFSC](https://www2.ifsc.usp.br/portal-ifsc/)), University of São Paulo ([USP](https://www5.usp.br/)). Based on Costa Didatic Text #17 ([CDT-17](https://www.researchgate.net/publication/337103890_Linear_Least_Squares_Versatile_Curve_and_Surface_Fitting_CDT-17)).

# Linear Least Squares

Simple and easy curve fitting for linearizable data. In other words, if we can represent it as a polynomial, it can be linearized with this method.

## Exercises

### 1 - Straight Lines

In [ ]:
def poly2D(X, P): 
    return sum([P[i] * X**i for i in range(len(P))], axis=0)

def straightfit(X, Y):
    if len(X)!=len(Y): raise
    At = np.array([[1]*len(X), X])
    A = np.transpose(At)
    return inv(At.dot(A)).dot(At.dot(Y))

X = array([-0.47, -0.26,  0.15,  0.82, -0.60])
Y = array([ 1.14,  1.21,  1.28,  1.47,  0.93])
P = straightfit(X, Y)
f = lambda X: poly2D(X, P)

fitplot2D(X, Y, f).draw();

### 2 - Polynomial curves 

In [ ]:
def polyfit(X, Y, o):
    if len(X)!=len(Y): raise
    At = np.array([[x**n for x in X] for n in range(o+1)])
    A = np.transpose(At)
    return inv(At.dot(A)).dot(At.dot(Y))

X = array([-0.47, -0.26,  0.15,  0.82, -0.60])
Y = array([ 0.12,  0.25,  0.18,  0.26,  -0.11])
P = polyfit(X, Y, 3)
f = lambda X: poly2D(X, P)

fitplot2D(X, Y, f).draw();

### 3 - Noise

In [ ]:
nf = lambda X, a, b: 1 - 0.5*X + 2*X**2 + rand(a, b) 

X = linspace(-1,1,100)
Y = array([nf(x, -0.3, 0.3) for x in X]) 
a,b,c = polyfit(X, Y, 2)
f = lambda X: a + b*X + c*X**2

fitplot2D(X, Y, f).draw();

### 4 - Exponential Curves

In [ ]:
f = lambda X: 0.5 * exp(2.5 * X)

X = randspace(0,1,5)
Y = f(X)
c, b = polyfit(X, log(Y), 1)
c = exp(c)
f = lambda X: c * exp(b * X)

fitplot2D(X, Y, f).draw();

### 4 - Multivariate polynomials

In [ ]:
def opoly(v, o):
    if len(v) == 1:
        p = []
        for n in range(o + 1):
            p.append(v[0]**n)
        return p
    p = []
    q = opoly(v[1:],o)
    for n in range(o + 1):
        for t in q:
            p.append(v[0]**n * t)
    return p

def opolyfit(data, order, vec_out):
    A = np.array([opoly(v, order) for v in data])
    At = np.transpose(A)
    return np.linalg.inv(At.dot(A)).dot(At.dot(vec_out))

def opolynomial(var, coef):
    p = opoly(var, int(len(coef)**(1. / len(var)) - 1))
    return sum([coef[i] * p[i] for i in range(len(coef))])

P = randspace(-1,1,9)
X = randspace(-1,1,50)
Y = randspace(-1,1,50)
Z = array([opolynomial(v, P) + rand(-0.1,0.1) for v in np.stack((X, Y), axis=1)])

Pf = opolyfit(transpose([X,Y]),2,Z)
f = lambda X, Y: opolynomial([X,Y],Pf)

fitplot3D(X,Y,Z,f,darkgraph(),ax3D())

## Project 1: Real World Data

### 1 - Linear case: Modelling my morning coffee watter

I use 250ml of watter every morning for making my coffee. The goal here is to predict how much time I should leave the watter inside the microwave to get it to my desired temperature. Turns out it's fairly linear. Here is how I did it:

In [ ]:
data = pd.read_csv("../input/microwave250gwatter/microwave.csv")

t, Ti, Tf = unbox(data, 't', 'Ti', 'Tf')

dT = Tf - Ti

X = array(t)
Y = array(dT)
P = straightfit(X, Y)
f = lambda X: poly2D(X, P)

(
    fitplot2D(X, Y, f) +
    p9.labels.ggtitle("$250ml$ of Watter on My Microwave") +
    p9.labels.ylab("Temperature Change (°C)") +
    p9.labels.xlab("Time inside Microwave (s)")
).draw();

formula = '$t \\approx \\frac{T_f - T_0 - %g}{%g}$' % (P[0],P[1])
plt.plot()
plt.text(83,55,formula,fontsize=14)
plt.show()

### 2 - Non linear case: Zipf's Law

There is a strange pattern that permeates texts in all existent languages. If we rank all words from most to least frequently appearing, we find the following relationship:

$$f_{(r)} \approx  \frac{f_1}{r} $$

Being $f_{(r)}$ the number of times a word of ranking $r$ appears, $f_1$ being the number of times the most common word appears. We can also write it like this:

$$f_{(r)} \approx  f_1 \times r^{-1} $$

Let us look at some data and verify if it approaches this result. We can see that the data appears linear when seen at log-log scale. So we can aply a simple linear least squares function using the logarithms of the frequencies and rankings and then convert it back to a power law.

In [ ]:
data = pd.read_csv('../input/english-word-frequency/unigram_freq.csv').head(1000)
print(data)

X = array(data.index)+1
Y = array(data['count'])
a, b = polyfit(log(X), log(Y), 1)
c = exp(a) ; 
f = lambda x: c * x**b

(
    fitplot2D(X, Y, f, res=50000) +
    p9.scales.scale_y_log10() +
    p9.scales.scale_x_log10() +
    p9.labels.ggtitle("Google's English Internet 1000 Most Used Words") +
    p9.labels.ylab("Number of Occurences") +
    p9.labels.xlab("Ranking (from most to least common)")
).draw();

formula = '$frequency \\approx (%.2g) \\times ranking^{%.2g}$ \n' % (c,b)
plt.plot()
plt.text(0.05,7.9,formula,fontsize=14)
plt.show()

### 3 - Multivariable case: Galton's Height Dataset and Regression to the Mean

Sir Francis Galton was a renown scientist that created the concept of correlation, the idea of "Regression Towards Mediocrity", and much more. In a particular research he put together a dataset of parents and adult children's heights to test if his regression theory worked. Here we will just linearize it to see a correlation between parents and children's heights. We will take a very simplified approach here, as the objective is just showcasing the LLS method.

In [ ]:
data = pd.read_table('../input/galton-height-data/galton-stata11.csv')

X = F = data['father']
Y = M = data['mother']
Z = H = data['height']

Pf = opolyfit(transpose([X,Y]),1,Z)
f = lambda X, Y: opolynomial([X,Y],Pf)
fig=darkgraph() ; ax=ax3D()
fitplot3D(X,Y,Z,f,fig,ax)
ax.set_xlabel("Father's Height")
ax.set_ylabel("Mother's Height")
ax.set_zlabel("Kid's Height");

Also, even without treating the data the way Gauton did originally, the regression effect is still visible when we apply linear least squares to the data. Note that mid-parental height is the term Gauton used to describe the mean of the parents  heights. In the fitted data, we can see that kids tend to be shorter if the parents have above average height, and taller if the parents are too short, sugesting that this population, if isolated, might regress to a mean height over many generations.

In [ ]:
Mean = np.mean([F, M], axis=0)
D = H - Mean
X = Mean ; Y = D

P = straightfit(X, Y)
f = lambda X: poly2D(X, P)
mean = pd.DataFrame({'x':[np.mean(Mean)],'y':[0],'Source':['Mean']})
(
    fitplot2D(X, Y, f) +
    p9.geom_point(p9.aes('x', 'y', color='Source'), data=mean, size=3) +
    p9.labels.ggtitle("Height difference between children and their parents") +
    p9.labels.ylab("Height Difference") +
    p9.labels.xlab("Mid-Parental Height") +
    p9.scale_color_manual(unbox(COLORS,"red", "blue", "orange"))
).draw();

It is hard to write about Gauton without remembering his theory of eugenics, that many years latter was used as basis for the nazist eugenics that culminated in the holocaust. A great reminder to ***always take statistics with a grain of salt***, please. 

I sugest reading the original paper from 1886 to take a look at Gauton's more detailed analisis, and just to contemplate a piece of science history, right at [this link](http://galton.org/essays/1880-1889/galton-1886-jaigi-regression-stature.pdf). 

Thanks for reading!